## Variable Selection

<a id='up'></a>

[Drop Weather](#drop-weather)

In [1]:
# Load libraries
import numpy as np 
import pandas as pd 
from subprocess import check_output
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import sys
#!{sys.executable} -m pip install xgboost

In [3]:
#from xgboost import XGBRegressor

In [4]:
import glob, re
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

In [5]:
np.random.seed(10)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

### 1.Data import and aggregation

In [6]:
# Data Aggregation

#train_wo = pd.read_csv('../input/train_without_orders.csv')
train_io = pd.read_csv('../input/train_in_orders.csv')
#test = pd.read_csv('../input/test.csv')

In [8]:
train_io.head()

,Unnamed: 0,visit_date,visitors,reserve_visitors_air,air_date_diff,air_genre_name,air_area_name,air_store_id,latitude,longitude,...,avg_wind_speed,avg_vapor_pressure,avg_humidity,avg_sea_pressure,avg_local_pressure,solar_radiation,cloud_cover,high_temperature,low_temperature,lon_plus_lat
0,0,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,air_ba937bf13d40fb24,35.658068,139.751599,...,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667
1,1,2016-01-13,21,NaN,NaN,Izakaya,Tōkyō-to Shinagawa-ku Higashigotanda,air_25e9888d30b386df,35.626568,139.725858,...,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.352426
2,2,2016-01-13,40,NaN,NaN,Izakaya,Tōkyō-to Minato-ku Shibakōen,air_fd6aac1043520e83,35.658068,139.751599,...,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667
3,3,2016-01-13,5,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,air_64d4491ad8cdb1c6,35.658068,139.751599,...,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667
4,4,2016-01-13,16,NaN,NaN,Other,Tōkyō-to Shibuya-ku Shibuya,air_5c65468938c07fa5,35.661777,139.704051,...,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.365828


In [9]:
train_io.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247009 entries, 0 to 247008
Data columns (total 35 columns):
Unnamed: 0              247009 non-null int64
visit_date              247009 non-null object
visitors                247009 non-null int64
reserve_visitors_air    26729 non-null float64
air_date_diff           26729 non-null float64
air_genre_name          247009 non-null object
air_area_name           247009 non-null object
air_store_id            247009 non-null object
latitude                247009 non-null float64
longitude               247009 non-null float64
cluster                 247009 non-null int64
prefecture              247009 non-null object
month                   247009 non-null int64
date                    247009 non-null int64
dw                      247009 non-null int64
dy                      247009 non-null int64
day_of_week             247009 non-null object
holiday_flg             247009 non-null int64
sunday                  247009 non-null int64
sat

  ### 1.1.Convert the date format 

#### One Hat Encoding function

In [10]:
def our_hat_code(myData, colQuery):
    temp = list(myData[colQuery].unique())
    print(temp)
    n = len(temp) 
    print(n)
    for ii in range(0,n):
        myData[temp[ii]] = None
        myData.loc[(myData[colQuery] == temp[ii] ),temp[ii]] = 1
        myData.loc[(myData[colQuery] != temp[ii]),temp[ii]] = 0

## <a id='drop-weather'>Drop weather</a>

[to the header](#up)

In [11]:
cptTrainInOrder = train_io.copy()
#len(cptTrainInOrder)

In [12]:
#cptTrainInOrder = cptTrainInOrder.drop(['hours_sunlight','avg_wind_speed','avg_vapor_pressure',\
 #                                       'avg_humidity','avg_sea_pressure','avg_local_pressure',\
  #                                      'cloud_cover','high_temperature','low_temperature','lon_plus_lat',\
  #                                      'solar_radiation'], axis=1)

In [13]:
# NEW FEATURES FROM Georgii Vyshnia
cptTrainInOrder['air_genre_name'] = cptTrainInOrder['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
cptTrainInOrder['air_area_name'] = cptTrainInOrder['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))

lbl = preprocessing.LabelEncoder()

for i in range(10):
    cptTrainInOrder['air_genre_name'+str(i)] = lbl.fit_transform(cptTrainInOrder['air_genre_name']\
                                                                 .map(lambda x:\
                                                                      str(str(x).split(' ')[i])\
                                                                      if len(str(x).split(' '))>i else ''))
    cptTrainInOrder['air_area_name'+str(i)] = lbl.fit_transform(cptTrainInOrder['air_area_name']\
                                                                .map(lambda x: str(str(x).split(' ')[i])\
                                                                     if len(str(x).split(' '))>i else ''))
    
cptTrainInOrder['air_genre_name'] = lbl.fit_transform(cptTrainInOrder['air_genre_name'])
cptTrainInOrder['air_area_name'] = lbl.fit_transform(cptTrainInOrder['air_area_name'])

In [14]:
#cptTrainInOrder.info()
cptTrainInOrder.head(2)

,Unnamed: 0,visit_date,visitors,reserve_visitors_air,air_date_diff,air_genre_name,air_area_name,air_store_id,latitude,longitude,...,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
0,0,2016-01-13,25,NaN,NaN,4,59,air_ba937bf13d40fb24,35.658068,139.751599,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-01-13,21,NaN,NaN,7,72,air_25e9888d30b386df,35.626568,139.725858,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(cptTrainInOrder.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247009 entries, 0 to 247008
Data columns (total 55 columns):
Unnamed: 0              247009 non-null int64
visit_date              247009 non-null object
visitors                247009 non-null int64
reserve_visitors_air    26729 non-null float64
air_date_diff           26729 non-null float64
air_genre_name          247009 non-null int32
air_area_name           247009 non-null int32
air_store_id            247009 non-null object
latitude                247009 non-null float64
longitude               247009 non-null float64
cluster                 247009 non-null int64
prefecture              247009 non-null object
month                   247009 non-null int64
date                    247009 non-null int64
dw                      247009 non-null int64
dy                      247009 non-null int64
day_of_week             247009 non-null object
holiday_flg             247009 non-null int64
sunday                  247009 non-null int64
satur

In [16]:
# One Hat Encoding air_genre_name
#our_hat_code(train_io,'air_genre_name')

In [17]:
# One Hat Encoding cluster
#train_io['cluster'] = train_io['cluster'].astype('category')
#our_hat_code(train_io,'cluster')

In [18]:
lbl = preprocessing.LabelEncoder()
cptTrainInOrder['cluster'] = lbl.fit_transform(cptTrainInOrder['cluster'])

cptTrainInOrder['cluster'].unique()

array([1, 3, 2, 7, 0, 5, 4, 6], dtype=int64)

In [19]:
# One Hat Encoding day_of_week
#our_hat_code(train_io,'day_of_week')

In [20]:
lbl = preprocessing.LabelEncoder()
cptTrainInOrder['day_of_week'] = lbl.fit_transform(cptTrainInOrder['day_of_week'])

In [21]:
des_width = 620
pd.set_option('display.width', des_width)
pd.set_option('display.max_columns', 200)
np.set_printoptions(linewidth=des_width)

In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; } </style>"))

In [23]:
cptTrainInOrder.tail(2)

,Unnamed: 0,visit_date,visitors,reserve_visitors_air,air_date_diff,air_genre_name,air_area_name,air_store_id,latitude,longitude,cluster,prefecture,month,date,dw,dy,day_of_week,holiday_flg,sunday,saturday,sat/sun/hol,calendar_date,precipitation,avg_temperature,hours_sunlight,avg_wind_speed,avg_vapor_pressure,avg_humidity,avg_sea_pressure,avg_local_pressure,solar_radiation,cloud_cover,high_temperature,low_temperature,lon_plus_lat,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
247007,247007,2016-01-11,22,NaN,NaN,2,20,air_71903025d39a4571,43.05546,141.340956,2,Hokkaidō,1,11,0,11,1,1,0,0,1.0,2016-01-11,1.5625,-8.228571,4.242857,2.785714,2.425,70.75,1014.7,1006.475,6.4875,8.075,-5.557143,-11.171429,184.396416,2,2,6,0,0,49,0,3,0,3,0,7,0,0,0,0,0,0,0,0
247008,247008,2016-01-11,3,3.0,3.0,7,19,air_7831b00996701c0f,43.07447,141.300889,2,Hokkaidō,1,11,0,11,1,1,0,0,1.0,2016-01-11,1.5625,-8.228571,4.242857,2.785714,2.425,70.75,1014.7,1006.475,6.4875,8.075,-5.557143,-11.171429,184.375359,7,2,0,0,0,49,0,2,0,1,0,6,0,0,0,0,0,0,0,0


In [24]:
#train_io.info()
#cptTrainInOrder.info()

In [25]:
#train_io = train_io.drop(['calendar_date' , 'prefecture', 'cluster', 'air_genre_name','air_area_name','day_of_week'], axis=1)

In [26]:
cptTrainInOrder= cptTrainInOrder.drop(['calendar_date' , 'prefecture'], axis=1)

In [27]:
#train_io.head()
#cptTrainInOrder.head()
#cptTrainInOrder.info()
cptTrainInOrder.columns

Index(['Unnamed: 0', 'visit_date', 'visitors', 'reserve_visitors_air', 'air_date_diff', 'air_genre_name', 'air_area_name', 'air_store_id', 'latitude', 'longitude', 'cluster', 'month', 'date', 'dw', 'dy', 'day_of_week', 'holiday_flg', 'sunday', 'saturday', 'sat/sun/hol', 'precipitation', 'avg_temperature', 'hours_sunlight', 'avg_wind_speed', 'avg_vapor_pressure', 'avg_humidity', 'avg_sea_pressure', 'avg_local_pressure', 'solar_radiation', 'cloud_cover', 'high_temperature', 'low_temperature', 'lon_plus_lat', 'air_genre_name0', 'air_area_name0', 'air_genre_name1', 'air_area_name1', 'air_genre_name2',
       'air_area_name2', 'air_genre_name3', 'air_area_name3', 'air_genre_name4', 'air_area_name4', 'air_genre_name5', 'air_area_name5', 'air_genre_name6', 'air_area_name6', 'air_genre_name7', 'air_area_name7', 'air_genre_name8', 'air_area_name8', 'air_genre_name9', 'air_area_name9'],
      dtype='object')

In [29]:
#train_io.info()
cptTrainInOrder.head(1)

,Unnamed: 0,visit_date,visitors,reserve_visitors_air,air_date_diff,air_genre_name,air_area_name,air_store_id,latitude,longitude,cluster,month,date,dw,dy,day_of_week,holiday_flg,sunday,saturday,sat/sun/hol,precipitation,avg_temperature,hours_sunlight,avg_wind_speed,avg_vapor_pressure,avg_humidity,avg_sea_pressure,avg_local_pressure,solar_radiation,cloud_cover,high_temperature,low_temperature,lon_plus_lat,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
0,0,2016-01-13,25,NaN,NaN,4,59,air_ba937bf13d40fb24,35.658068,139.751599,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667,4,7,7,3,0,26,0,4,0,75,0,0,0,0,0,0,0,0,0,0


Now all data is converted to the format of the number.

### Preparation and separation of data forvVariable selection

In [30]:
#varSel = pd.DataFrame({'Variable': train_io.columns[6:58]})
#varSel.head(58)

In [31]:
#vv = cptTrainInOrder.columns.
#vv2 = cptTrainInOrder.columns[5]
#print(vv2)
#varSel1 = pd.DataFrame({'Variable': cptTrainInOrder.columns.index})
 
#varSel1.head(2)

In [32]:
#vv3 = cptTrainInOrder.columns
#print(vv3)
#varSel2 = pd.DataFrame({'Variable': cptTrainInOrder.columns.index})
tt6 = ['air_genre_name', 'air_area_name','latitude', 'longitude', 'cluster', 'month', 'date', 'dw', 'dy', 'day_of_week',\
       'holiday_flg', 'sunday', 'saturday', 'sat/sun/hol', 'precipitation', 'avg_temperature',\
       'hours_sunlight', 'avg_wind_speed', 'avg_vapor_pressure', 'avg_humidity', 'avg_sea_pressure',\
       'avg_local_pressure', 'solar_radiation', 'cloud_cover', 'high_temperature', 'low_temperature',\
       'lon_plus_lat', 'air_genre_name0', 'air_area_name0', 'air_genre_name1', 'air_area_name1', 'air_genre_name2',
       'air_area_name2', 'air_genre_name3', 'air_area_name3', 'air_genre_name4', 'air_area_name4', 'air_genre_name5', 'air_area_name5',\
       'air_genre_name6', 'air_area_name6', 'air_genre_name7', 'air_area_name7', 'air_genre_name8', 'air_area_name8', 'air_genre_name9', 'air_area_name9'] 

varSel7 = pd.DataFrame({'Variable': tt6})
varSel7.head(50)

,Variable
0,air_genre_name
1,air_area_name
2,latitude
3,longitude
4,cluster
5,month
6,date
7,dw
8,dy
9,day_of_week


In [33]:
#train_io = train_io.dropna()

In [34]:
cptTrainInOrder = cptTrainInOrder.dropna()

In [35]:
## remove unnecessary vars
#X = train_io.loc[:,train_io.columns[5:31]]
#X.head()

In [36]:
## remove unnecessary vars
X5 = cptTrainInOrder.loc[:,tt6]
X5.head()

,air_genre_name,air_area_name,latitude,longitude,cluster,month,date,dw,dy,day_of_week,holiday_flg,sunday,saturday,sat/sun/hol,precipitation,avg_temperature,hours_sunlight,avg_wind_speed,avg_vapor_pressure,avg_humidity,avg_sea_pressure,avg_local_pressure,solar_radiation,cloud_cover,high_temperature,low_temperature,lon_plus_lat,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
26,6,71,35.661777,139.704051,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.365828,6,7,2,3,0,38,0,4,0,76,0,0,0,0,0,0,0,0,0,0
30,8,59,35.658068,139.751599,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667,8,7,9,3,0,26,0,4,0,75,0,0,0,0,0,0,0,0,0,0
39,6,59,35.658068,139.751599,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667,6,7,2,3,0,26,0,4,0,75,0,0,0,0,0,0,0,0,0,0
40,6,59,35.658068,139.751599,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.409667,6,7,2,3,0,26,0,4,0,75,0,0,0,0,0,0,0,0,0,0
49,7,71,35.661777,139.704051,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.365828,7,7,0,3,0,38,0,4,0,76,0,0,0,0,0,0,0,0,0,0


In [37]:
#y = train_io.loc[:,train_io.columns[2]]
#print([X.shape,y.shape])

In [38]:
cptTrainInOrder.head(1)

,Unnamed: 0,visit_date,visitors,reserve_visitors_air,air_date_diff,air_genre_name,air_area_name,air_store_id,latitude,longitude,cluster,month,date,dw,dy,day_of_week,holiday_flg,sunday,saturday,sat/sun/hol,precipitation,avg_temperature,hours_sunlight,avg_wind_speed,avg_vapor_pressure,avg_humidity,avg_sea_pressure,avg_local_pressure,solar_radiation,cloud_cover,high_temperature,low_temperature,lon_plus_lat,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
26,26,2016-01-13,14,2.0,0.0,6,71,air_f911308e19d64236,35.661777,139.704051,1,1,13,2,13,6,0,0,0,0.0,0.0,3.542857,8.833333,1.514286,4.9,60.0,1013.1,1010.1,10.86,2.5,9.171429,-2.028571,175.365828,6,7,2,3,0,38,0,4,0,76,0,0,0,0,0,0,0,0,0,0


In [39]:
y1 = cptTrainInOrder.loc[:,cptTrainInOrder.columns[2]]
print([X5.shape,y1.shape])

[(22265, 47), (22265,)]


### Variable Selection using LASSO

In [40]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

lassomod2 = Lasso(alpha=0.1).fit(X5, y1)

In [41]:
model2 = SelectFromModel(lassomod2, prefit=True)
model2.get_support()

array([ True,  True,  True, False,  True, False,  True,  True,  True,  True,  True, False,  True, False,  True, False,  True,  True,  True,  True,  True,  True,  True, False,  True,  True, False,  True,  True,  True, False,  True,  True, False,  True, False,  True, False, False, False,  True, False, False, False, False, False, False])

In [42]:
varSel7['Lasso'] = model2.get_support().astype('int64')
varSel7

,Variable,Lasso
0,air_genre_name,1
1,air_area_name,1
2,latitude,1
3,longitude,0
4,cluster,1
5,month,0
6,date,1
7,dw,1
8,dy,1
9,day_of_week,1


## Variable Selection using Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rfmod2 = RandomForestClassifier().fit(X5, y1)

C:\Users\gali\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [44]:
model3 = SelectFromModel(rfmod2, prefit=True)
model3.get_support()

array([ True,  True,  True,  True, False,  True,  True, False,  True, False, False, False, False, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, False,  True, False, False, False, False, False, False,  True, False, False, False, False, False, False, False, False, False, False])

In [45]:
varSel7['RandomForest'] = model3.get_support().astype('int64')
varSel7

,Variable,Lasso,RandomForest
0,air_genre_name,1,1
1,air_area_name,1,1
2,latitude,1,1
3,longitude,0,1
4,cluster,1,0
5,month,0,1
6,date,1,1
7,dw,1,0
8,dy,1,1
9,day_of_week,1,0


### Variable Selection using Gradient Boosting classification

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

In [47]:
gbmod2 = GradientBoostingClassifier().fit(X5, y1)

In [48]:
model4 = SelectFromModel(gbmod2, prefit=True)
model4.get_support()

array([ True,  True,  True,  True, False, False,  True, False,  True, False, False, False, False, False,  True,  True,  True,  True,  True,  True,  True,  True,  True, False,  True,  True,  True,  True, False,  True, False, False,  True, False, False, False,  True, False, False, False,  True, False, False, False, False, False, False])

In [ ]:
varSel7['GradientBoost'] = model4.get_support().astype('int64')
varSel7

### Summarization and Selection of Variables 

In [ ]:
#varSel6= varSel6.drop('sum2',axis=1)
varSel7['sum'] =  np.sum(varSel7,axis=1)
varSel7

In [ ]:
varSel7.groupby('sum')['Variable'].count()

In [ ]:
result111 = varSel7[varSel7['sum'] == 1]

In [ ]:
result111

In [ ]:
result10 = varSel7[varSel7['sum'] == 0]
result10

In [ ]:
result12 = varSel7[varSel7['sum'] == 2]
result12

In [ ]:
result13 = varSel7[varSel7['sum'] == 3]
result13